# 05. First Trial with Intuition of MCTS

## 환경설정

오목 학습에 필요한 모듈들을 모두 불러옵니다.

In [1]:
!rm -rf *
!git clone https://github.com/lumiknit/mock5.py
!mv mock5.py/mock5 .
!mv mock5.py/gen-record/gen.cpp .
!g++ -O2 -fopenmp -o gen gen.cpp

Cloning into 'mock5.py'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 89 (delta 36), reused 88 (delta 35), pack-reused 0
Unpacking objects: 100% (89/89), done.


In [2]:
from mock5 import Mock5
from mock5.analysis import Analysis as M5Analysis
from mock5.agent_random import agent as m5agent_random
from mock5.agent_analysis_based import agent as m5agent_a

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: {}".format(device))

Device: cuda


## 컨셉

- Model `f`는 오목판을 해당 상태로 전이하면 이길 확률을 출력합니다.
(즉, 오목판을 입력을 받아서 하나의 스칼라를 출력합니다.)
- 학습을 시킬 때 이전에 TicTacToe에서 MCTS한 것처럼 각 상태에 대해 승률을 누적합니다.
- `f`는 오목판을 평가하는 함수의 일종인데,
오목판에서는 연결될 가능성이 높은 조각들이 고평가가 됩니다.
이러한 연결 성분을 먼저 선정하는 것이 중요할 것 같기에
Convoluitonal Layer를 사용합니다.

In [3]:
# Board Size
W, H = 11, 11

In [13]:
import os
def gen_data(size, rnd):
  filename = "temp.data"
  os.system("./gen {} {} {} {} {}".format(filename, size, H, W, rnd))
  X = []
  Y = []
  with open(filename) as file:
    left, v, w = 0, 0, 1
    for line in file:
      if left <= 0:
        z = [torch.ones(H, W, dtype=torch.float),
             torch.zeros(H, W, dtype=torch.float),
             torch.zeros(H, W, dtype=torch.float)]
        bd_my = torch.stack(z)
        bd_op = bd_my.clone()
        v, left = map(int, line.split())
        w = 1 if v == 1 else -1
      else:
        idx = int(line)
        y, x = idx // W, idx % W
        # Adjust my board
        bd_my[0][y][x] = 0
        bd_my[v][y][x] = 1
        X.append(bd_my.clone())
        Y.append(torch.tensor([w], dtype=torch.float))
        # Adjust op board
        bd_op[0][y][x] = 0
        bd_op[3 - v][y][x] = 1
        X.append(bd_op.clone())
        Y.append(torch.tensor([-w], dtype=torch.float))
        # Iter
        v = 3 - v
        left -= 1
  Xs = torch.stack(X)
  Ys = torch.stack(Y)
  return Xs, Ys

## Nueral Network

In [5]:
class Flatten(torch.nn.Module):
  def forward(self, x):
    return x.flatten(1, -1)

In [28]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 5, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 128, 5, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(128, 128, 3, padding='same'),
    nn.ReLU(),
    Flatten(),
    nn.Linear(128 * 2 * 2, 1)
).to(device)

loss_fn = nn.MSELoss()

In [27]:
def learn(n_epoch, batch_size, repeat_for_batch):
  learning_rate = 1e-2
  optimizer = optim.SGD(
      model.parameters(),
      lr=learning_rate,
      momentum=0.5,
      weight_decay=1e-6,
      nesterov=True,
  )

  epsilon = 0.1
  log_int = max(1, int(repeat_for_batch / 10))
  
  acc_n = 0
  acc_loss = 0

  for epoch in range(n_epoch):
    print("-- Epoch {} --".format(epoch))
    X, Y = gen_data(batch_size, epsilon)
    X = X.to(device)
    Y = Y.to(device)

    for i in range(repeat_for_batch):
      optimizer.zero_grad()
      Y_ = model(X)
      loss = loss_fn(Y_, Y)
      loss.backward()
      optimizer.step()

      acc_loss += loss.item()
      acc_n += 1

      if i % log_int == log_int - 1 or i == repeat_for_batch - 1:
        print("E {:.3f}: Loss = {:.6f}".format(
            epoch + (i + 1) / repeat_for_batch, acc_loss / acc_n
        ))
        acc_loss = 0
        acc_n = 0

In [29]:
torch.cuda.empty_cache()
learn(20, 1000, 100)

-- Epoch 0 --
E 0.100: Loss = 1.002199
E 0.200: Loss = 0.997579
E 0.300: Loss = 0.995609
E 0.400: Loss = 0.993733
E 0.500: Loss = 0.991761
E 0.600: Loss = 0.989675
E 0.700: Loss = 0.987397
E 0.800: Loss = 0.984819
E 0.900: Loss = 0.982009
E 1.000: Loss = 0.978791
-- Epoch 1 --
E 1.100: Loss = 0.975954
E 1.200: Loss = 0.971606
E 1.300: Loss = 0.966454
E 1.400: Loss = 0.960361
E 1.500: Loss = 0.953173
E 1.600: Loss = 0.944580
E 1.700: Loss = 0.934098
E 1.800: Loss = 0.921242
E 1.900: Loss = 0.905402
E 2.000: Loss = 0.885707
-- Epoch 2 --
E 2.100: Loss = 0.872796
E 2.200: Loss = 0.846548
E 2.300: Loss = 0.814511
E 2.400: Loss = 0.775839
E 2.500: Loss = 0.730481
E 2.600: Loss = 0.679816
E 2.700: Loss = 0.634578
E 2.800: Loss = 0.810228
E 2.900: Loss = 0.574819
E 3.000: Loss = 0.526955
-- Epoch 3 --
E 3.100: Loss = 0.588199
E 3.200: Loss = 0.578783
E 3.300: Loss = 0.484657
E 3.400: Loss = 0.468277
E 3.500: Loss = 0.455918
E 3.600: Loss = 0.446370
E 3.700: Loss = 0.496265
E 3.800: Loss = 0.4

KeyboardInterrupt: ignored

## Evaluation

In [9]:
def agent_model(game):
  player = game.player
  Xs = []
  idxs = []
  for i in range(H * W):
    if game.place_stone_at_index(i):
      idxs.append(i)
      Xs.append(game.tensor(player=player, dtype=torch.float))
      game.undo()
  X = torch.stack(Xs).to(device)
  with torch.no_grad():
    y = model(X)
  j = torch.argmax(y)
  k = idxs[j]
  return (k // W), (k % W)
    

In [30]:
def test_agents(num_game, agent1, agent2):
  w1 = 0
  w2 = 0
  for i in range(num_game):
    g = Mock5(H, W)
    result = g.play(agent1, agent2, print_intermediate_state=False, print_messages=False)
    if result == 1: w1 += 1
    elif result == 2: w2 += 1
  print("-- Test Result --")
  print("Agent1 = {} / Agent2 = {}".format(agent1, agent2))
  print("Total: {}".format(num_game))
  print("A1 Win: {} ({:.3f})".format(w1, w1 / num_game))
  print("A2 Win: {} ({:.3f})".format(w2, w2 / num_game))

In [33]:
test_agents(20, m5agent_random, agent_model)

-- Test Result --
Agent1 = <function agent at 0x7f226b3fd290> / Agent2 = <function agent_model at 0x7f21a1785950>
Total: 20
A1 Win: 2 (0.100)
A2 Win: 18 (0.900)


In [32]:
Mock5(H, W).play(m5agent_random, agent_model)

 [ Turn   0 ; 1P's turn (tone = O) ]
  | 0 1 2 3 4 5 6 7 8 9 A
--+----------------------
0 | . . . . . . . . . . .
1 | . . . . . . . . . . .
2 | . . . . . . . . . . .
3 | . . . . . . . . . . .
4 | . . . . . . . . . . .
5 | . . . . . . . . . . .
6 | . . . . . . . . . . .
7 | . . . . . . . . . . .
8 | . . . . . . . . . . .
9 | . . . . . . . . . . .
A | . . . . . . . . . . .
 [ Turn   1 ; 2P's turn (tone = X) ]
  | 0 1 2 3 4 5 6 7 8 9 A
--+----------------------
0 | . . . . . . . . . . .
1 | . . . . . . . . . . .
2 | . . . . . . . . . . .
3 | . . . . . . . . . . .
4 | . . . . . . . . . . .
5 | . . . . . O . . . . .
6 | . . . . . . . . . . .
7 | . . . . . . . . . . .
8 | . . . . . . . . . . .
9 | . . . . . . . . . . .
A | . . . . . . . . . . .
 [ Turn   2 ; 1P's turn (tone = O) ]
  | 0 1 2 3 4 5 6 7 8 9 A
--+----------------------
0 | . . . . . . . . . . .
1 | . . . . . . . . . . .
2 | . . . . . . . X . . .
3 | . . . . . . . . . . .
4 | . . . . . . . . . . .
5 | . . . . . O . . . . .
6 | .

2

## Conclusion

- Google Colab의 VRAM이 약 16GB정도 되는데,
현재 약 1000 게임을 batch로 돌리고 있으며,
이는 대량 50k개 정도입니다.
하지만 이보다 몇배 늘리게 되면 RAM이 부족하다고 나오는 것으로 보아
현 상황에서는 batch size를 더 늘리는 것은 좋지 않아 보입니다.
- Learning rate와 별개로 momentum을 0.9 정도로 크게 주는
경우에 average loss가 0.5까지 줄어들다가 다시 1.5로
튀어오르는 문제가 있습니다. Loss space가 생각보다 굴곡이 적어서
생기는 문제로 보여서, 이후에 학습을 시킬 때에도
momentum을 크게 주지 않는 것이 좋아보입니다.
- 이전의 Q-Function에 대한 nueral network는 전혀 수렴하는
느낌이 없었지만, 그에 비해 이 방법은 (적어도 초기에는) 명확하게
수렴하는 모습이 보여집니다. Q-Learning의 경우에는 마지막 보상이
주어지는 수를 제외하면 자신의 추측과 실제 결과를 바탕으로 중간에
수렴을 시켜야하는 과정이 많은데, 이 부분에서 수렴이 잘 되지 않는
문제가 있지 않은가 싶습니다.
- 현재 학습된 모델을 보면 특정 방향으로 돌이 연결되어야 점수가 잘
나온다는 것을 약간 이해한 듯한 경향을 보입니다. 다만 길게 연결하는 것이 짧게 여러개 만드는 것보다 좋다는 것은 그다지 인식하지 못하는 것 같아
그 부분은 고려해 두는 것이 좋겠습니다.
- (TODO1) 현재는 게임을 진행한 뒤, 그 게임을 그대로 넣습니다.
하지만 실제로는 $D_4$-group에 따른 변환에도 대칭/회전을
바탕으로 똑같은 판단을 할 수 있으며, 제대로 된 경우에는
똑같은 판단을 해야합니다. 때문에 단순히 게임을 한 방향으로
넣는 것이 아니라 $D_4$-group에 맞춰서 변환하는 것이 필요해보입니다.
- 다만 translation이 얼마나 필요할지는 모르겠는데, CNN의 경우
적당한 구간을 나눠서 해당 범위에 대해 판단을 하기 때문에, 
translation이 있어도 비교적 쉽게 패턴을 찾아내는 것은 가능할
것입니다.
- (TODO1) 만약 이런 방식으로 일정 기보를 주고 학습을 할 경우에는
기보가 다양할 필요가 있습니다.
예를 들어서 3-4 같은 특수한 형태가 매우 유리하다든가
4목이 있는 경우 수를 이어서 게임을 끝내는 등의 상황이 있겠습니다.
- (TODO2) Q-function과 현재 방법의 가장 큰 차이인데,
Q-function은 현재 상태와 행동에 대해 평가를 내리는 함수지만,
위 NN은 특정 상태에 대해서만 평가를 내리는 함수입니다.
오목은 궁극적으로는 임의의 상태에 대한 가치를 판단할 수 있으면
행동이 가능하기 때문에 이 둘은 학습이 완료되면 비슷한 성능을
보이겠지만, 학습하는 과정에서는 조금 다릅니다.
Q-function의 경우에는 같은 상태로 전이되더라도 최악의 상황에서
최선의 수를 두는 경우와 최선의 상황에서 최악의 수를 두는 경우가
구별이 되어 별도의 보상을 주는 것이 가능하지만,
위 학습 방식에서는 이 둘을 구분하지 못합니다.
이를 해결하려면 Q-learning처럼 미래의 상태를 바탕으로 현재의
상태에 대한 평가를 조절하는 것이 필요한데, 이 때문에
이후 부분적으로 Q-learning을 적용할 필요가 있을 것입니다.
- (TODO3) 위와 비슷한 내용인데, 현재는 각 상태에 대해
동일한 값의 라벨을 부여했습니다. (만약 최종적으로 자신이 이기면
모두 1로, 최종적으로 자신이 지면 모두 -1로)
만약 위 방법이 MCTS를 구현하기에 적절히 복잡하다면 이 방법을
반복하여서 MCTS로 근사하는 것이 가능하겠지만, 실제 수렴 속도가
얼마나 될지 모릅니다. 따라서 기보를 제공한다면, 각 수에 대한
평가치를 연속적으로 조절할 필요가 있을 것입니다.
예를 들어서 게임을 빨리 끝내는 것이 유리하므로, 이러한 게임에
가산점을 줄 수 있을 것이고, 흔히 수가 제한되어서 무조건
막아야만 하는 수가 있는 경우 점수를 강제로 깎는 식이 있을 것입니다.
- (TODO4) 각 위치별 평가는 해당 근방의 돌의 배치에 의해 결정이 되며, 판이 아주 큰 경우가 아니면 두개의 인접한 근방은 두 범위에
모두 공통된 돌들 외에는 큰 영향을 받지 않을 것이라고 추측해볼 수
있습니다. (예를 들어서 C자 모양의 neighborhood cover가
있고, C의 끊어진 쪽의 두 neighborhood가 있을 때 이
C자를 따라서 한바퀴 돌아서 반대편에 영향을 주는 것은
힘들 것이라는 추측)
그렇다면 마지막 layer가 linear layer로 학습하기보다는
단순히 sum 등으로 전부 합하는 것으로도 충분하지 않을가 생각해볼
수 있는데, 이를 비교해보는 것이 도움이 될 것이라 생각합니다.

### TODO1

- 기보를 생성하는 알고리즘 외에도 흔히 볼 수 있는 일부
유형들을 제공하도록 합니다.
- 기보 자체를 성질을 유지하는 변환으로 변환하여 학습에 사용합니다.

### TODO2

- 현 방식에 Q-learning을 부분적으로 적용합니다.

### TODO3

- 기보의 각 수에 대해 평가를 내려서 학습의 label로 사용합니다.

### TODO4

- Dense neural network 부분을 조절해봅니다.